In [ ]:
from MonteCarlo import MonteCarlo
import matplotlib.pyplot as plt
from scipy import constants as cst
import numpy as np

In [ ]:
self = MonteCarlo(maximum_steps=1000,
    desired_mu=-10,
    dumping_period=1000,
    thermo_period=100,
    number_atoms=[200, 200],
    box_dimensions=[20, 20, 20],
    epsilon=[0.1, 0.1],
    sigma = [3, 6],
    atom_mass = [10, 10])
self.run()

In [ ]:
self.neighbor

In [ ]:
cst.centi/cst.angstrom

In [ ]:
import sys, os, git, time

import MDAnalysis as mda
from MDAnalysis.analysis import distances

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/python-codes/")

from MinimizeEnergy import MinimizeEnergy
from MonteCarlo import MonteCarlo
from MolecularDynamics import MolecularDynamics

import numpy as np

In [ ]:
ti = time.time()

me = MinimizeEnergy(maximum_steps=10,
    number_atoms=[50],
    epsilon=[0.1],
    sigma = [3],
    atom_mass = [1],
    box_dimensions=[20, 20, 20],
    neighbor=10,
    thermo = 2,
    cut_off = 9,
    dump = 2)
me.perform_energy_minimization()

tf = time.time()

print("time =", np.round(tf-ti,2), "s")

In [ ]:
ti = time.time()

self = MolecularDynamics(maximum_steps = 10000,
    number_atoms = [20],
    epsilon = [0.1],
    sigma = [3],
    atom_mass = [1],
    tau_temp=100,
    box_dimensions=[20, 20, 20],
    neighbor = 10,
    thermo = 50,
    cut_off = 9,
    dump = 50)
self.run()

tf = time.time()

print("time =", np.round(tf-ti,2), "s")

In [ ]:
np.shape(self.evaluate_rij_matrix())

In [ ]:
r = np.linalg.norm(self.evaluate_rij_matrix(), axis=2)

In [ ]:
help(np.histogram)